In [1]:
!pip install streamlit

In [2]:
!pip install pyngrok

In [10]:
%%writefile app.py
import streamlit as st
from PIL import Image
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
from tensorflow.keras.preprocessing.image import img_to_array
#from tensorflow.keras.applications import preprocess_input


st.set_page_config(page_title="Multiclass Fish Image Classification")

class_labels = [
      'animal fish',
      'animal fish bass',
      'fish sea_food black_sea_sprat',
      'fish sea_food gilt_head_bream',
      'fish sea_food hourse_mackerel',
      'fish sea_food red_mullet',
      'fish sea_food red_sea_bream',
      'fish sea_food sea_bass',
      'fish sea_food shrimp',
      'fish sea_food striped_red_mullet',
      'fish sea_food trout'
]

def get_tp_fp_tn_fn(cm, class_labels):
    metrics = []
    for i in range(len(class_labels)):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)

        metrics.append({
            "Class": class_labels[i],
            "TP": TP,
            "FP": FP,
            "TN": TN,
            "FN": FN
        })
    return metrics

# 🎯 Sidebar Navigation
st.sidebar.title("🧭 Navigation")
section = st.sidebar.radio("Choose Dashboard Section", ["CNN Model Evaluation", "Transfer Learning Model Comparison", "Fish Species Classifier"])

# ─────────────────────────────────────────────────────
# 📊 SECTION 1: Individual Model Evaluation Dashboard
if section == "CNN Model Evaluation":
    st.title("📊 CNN Model Evaluation Dashboard")

    # Load single model history
    try:
        with open("reports/training_history.json", "r") as f:
            history = json.load(f)
    except:
        st.error("reports/training_history.json not found.")

    # Load prediction results
    try:
        df = pd.read_csv("predictions/CNN_test_predictions.csv")
        class_labels = sorted(df['True_Label'].unique())
    except:
        st.error("predictions/CNN_test_predictions.csv not found.")
        st.stop()

    # Subnavigation
    st.sidebar.header("🔎 Evaluation Tabs")
    tab = st.sidebar.radio("View", ["Training Curves", "Confusion Matrix", "Prediction Explorer"])

    if tab == "Training Curves":
        st.subheader("📈 Accuracy & Loss")
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))

        axs[0].plot(history['accuracy'], label='Train')
        axs[0].plot(history['val_accuracy'], label='Val')
        axs[0].set_title("Accuracy")
        axs[0].legend()

        axs[1].plot(history['loss'], label='Train')
        axs[1].plot(history['val_loss'], label='Val')
        axs[1].set_title("Loss")
        axs[1].legend()

        st.pyplot(fig)

    elif tab == "Confusion Matrix":
        st.subheader("📉 Confusion Matrix")

        y_true = df['True_Label']
        y_pred = df['Predicted_Label']
        cm = confusion_matrix(y_true, y_pred, labels=class_labels)
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels, ax=ax)
        plt.xlabel("Predicted")
        plt.ylabel("True")
        st.pyplot(fig)

        acc = accuracy_score(y_true, y_pred)
        st.markdown(f"**🔍 Overall Accuracy:** `{acc:.2%}`")

    # elif tab == "Misclassified Samples":
    #    st.subheader("🧐 Misclassified Samples")
    #    incorrect = df[df["True_Label"] != df["Predicted_Label"]]
    #
    #    if incorrect.empty:
    #       st.success("No misclassifications! 🎉")
    #   else:
    #       sample = incorrect.sample(min(len(incorrect), 9), random_state=42)
    #       for i, row in sample.iterrows():
    #           image_path = os.path.join("C:/Users/conta/OneDrive/Desktop/Multiclass/data/test", row["Image"])
    #           if os.path.exists(image_path):
    #               img = Image.open(image_path)
    #               st.image(img, caption=f"True: {row['True_Label']} | Predicted: {row['Predicted_Label']}", use_container_width=True)
    #           else:
    #               st.warning(f"Image not found: {row['Image']}")

    elif tab == "Prediction Explorer":
        st.subheader("📂 Prediction Explorer")
        st.dataframe(df)
        csv = df.to_csv(index=False).encode('utf-8')
        st.download_button("📥 Download CSV", data=csv, file_name='test_predictions.csv', mime='text/csv')


# ─────────────────────────────────────────────────────
# 🏆 SECTION 2: Transfer Learning Leaderboard Dashboard
elif section == "Transfer Learning Model Comparison":
    st.title("🏆 Transfer Learning Model Comparison")

    # Load multi-model results
    try:
        with open("reports/model_training_results.json", "r") as f:
            results = json.load(f)
    except:
        st.error("reports/model_training_results.json not found.")
        st.stop()

    model_names = list(results.keys())
    st.sidebar.header("📋 Compare Models")
    selected = st.sidebar.multiselect("Select models", model_names, default=model_names)

    if selected:
        st.subheader("📈 Validation Accuracy & Loss")
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        for name in selected:
            axs[0].plot(results[name]['val_accuracy'], label=name)
        axs[0].set_title("Accuracy")
        #axs[0].legend(loc='upper right', bbox_to_anchor=(1.2, 1))

        for name in selected:
            axs[1].plot(results[name]['val_loss'], label=name)
        axs[1].set_title("Loss")
        axs[1].legend(loc='upper right', bbox_to_anchor=(1.5, 1))

        st.pyplot(fig)

        summary = []
        for name in selected:
            acc = round(results[name]["val_accuracy"][-1], 4)
            loss = round(results[name]["val_loss"][-1], 4)

            # Load predictions from each model's prediction file
            pred_path = os.path.join("predictions", f"{name}_test_predictions.csv")
            if os.path.exists(pred_path):
                df_pred = pd.read_csv(pred_path)
                y_true = df_pred["True_Label"]
                y_pred = df_pred["Predicted_Label"]

                precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
                recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)
                f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
                cm = confusion_matrix(y_true, y_pred)

                confusion_breakdown = get_tp_fp_tn_fn(cm, class_labels)
                # Sum up TP, FP, TN, FN across all classes
                TP_total = sum(row["TP"] for row in confusion_breakdown)
                FP_total = sum(row["FP"] for row in confusion_breakdown)
                TN_total = sum(row["TN"] for row in confusion_breakdown)
                FN_total = sum(row["FN"] for row in confusion_breakdown)



                #st.write(cm)
                summary.append({
                    "Model": name,
                    "Validation Accuracy": acc,
                    "Validation Loss": loss,
                    "Precision": round(precision, 3),
                    "Recall": round(recall, 3),
                    "F1-Score": round(f1, 3),
                    "TP": TP_total,
                    "FP": FP_total,
                    "TN": TN_total,
                    "FN": FN_total
                    })

            else:
                st.warning(f"Prediction file not found: {pred_path}")

        if summary:
            df_summary = pd.DataFrame(summary).sort_values("Validation Accuracy", ascending=False).reset_index(drop=True)
            st.subheader("📋 Final Performance Summary")
            st.dataframe(df_summary)

            #for model_summary in summary:
            #  model_name = model_summary["Model"]
            #  class_breakdown = model_summary["Class Breakdown"]
            #
            #  st.subheader(f"📊 TP/FP/TN/FN Breakdown – {model_name}")
            #  df_breakdown = pd.DataFrame(class_breakdown)
            #  st.dataframe(df_breakdown)

            # Number of columns per row
            cols_per_row = 3
            rows = [selected[i:i + cols_per_row] for i in range(0, len(selected), cols_per_row)]

            for row_models in rows:
                cols = st.columns(len(row_models))
                for idx, name in enumerate(row_models):
                    cm_path = os.path.join("reports", f"{name}_confusion_matrix.png")
                    if os.path.exists(cm_path):
                        cols[idx].image(cm_path, caption=f"{name}", use_container_width = True)
                    else:
                        cols[idx].warning(f"Image not found for: {name}")

        else:
            st.warning("No valid prediction results available.")


        best_model = df_summary.iloc[0]["Model"]
        st.success(f"🥇 Best Performing Model: **{best_model}**")

        # Download CSV
        csv = df_summary.to_csv(index=False).encode("utf-8")
        st.download_button("📥 Download Leaderboard CSV", data=csv, file_name="cnn_leaderboard.csv", mime="text/csv")

        # Model Insights
        with st.expander("📎 Model Insights"):
            st.markdown("""
            - **EfficientNetB0** → ✨ High performance + low loss.
            - **MobileNetV2** → ⚡ Lightweight yet accurate.
            - **ResNet50** → 📉 Stable classic choice.
            - **VGG16** → 🧱 Deep but heavier.
            - **InceptionV3** → 🎯 Needs data tuning.
            - **CNN** → 🎯 Solid Overall.
            """)
    else:
        st.warning("Please select at least one model.")

### Section 3: Fish Species Classifier
elif section == "Fish Species Classifier":
    st.title("🐟 Fish Species Classification")

    # Select model to use for prediction
    try:
        with open("reports/model_training_results.json", "r") as f:
            results = json.load(f)
        model_names = list(results.keys())
        best_model = sorted(model_names, key=lambda name: results[name]["val_accuracy"][-1], reverse=True)[0]
        #st.write(best_model)
    except:
        st.error("reports/model_training_results.json not found.")
        st.stop()

    h5_path = os.path.join("models", f"{best_model.lower()}_finetuned.h5")
    if not os.path.exists(h5_path):
        st.error(f"Best model file not found: {h5_path}")
        st.stop()

    model = load_model(h5_path)

    # Load class labels from test folder
   # test_dir = "C:/Users/conta/OneDrive/Desktop/Multiclass/data/test"
    #test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    #    test_dir, target_size=(224, 224), batch_size=1, class_mode="categorical", shuffle=False
    #)


    st.sidebar.header("🖼️ Upload Fish Image")
    uploaded_file = st.sidebar.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])

    if uploaded_file:
        img = Image.open(uploaded_file).convert("RGB")
        col1, col2 = st.columns([1, 3])
        with col1:
          st.image(img, caption="Uploaded Fish Image", width=150)
        with col2:
          # Preprocess for model
          img_resized = img.resize((224, 224))
          x = img_to_array(img_resized)
          x = np.expand_dims(x, axis=0)
          #x = preprocess_input(x)

          # Predict
          preds = model.predict(x)
          top_idx = np.argmax(preds)
          top_label = class_labels[top_idx]
          confidence = preds[0][top_idx]

          st.markdown(f"**🐟 Predicted Species:** `{top_label}`")
          st.markdown(f"**🎯 Confidence Score:** `{confidence:.2%}`")

        # Show all probabilities (optional)
        with st.expander("🔎 See all class probabilities"):
            prob_df = pd.DataFrame({
                "Species": class_labels,
                "Confidence": (preds[0] * 100).round(2)
            }).sort_values("Confidence", ascending=False).reset_index(drop=True)
            st.dataframe(prob_df)



Overwriting app.py


In [4]:
!npm install localtunnel

⠙⠹⠸⠼
up to date, audited 60 packages in 874ms
⠴
⠴5 packages are looking for funding
⠴  run `npm fund` for details
⠴
6 vulnerabilities (1 low, 2 moderate, 3 high)

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

1. Include CNN Augmented to the Pre-trained model comparison
2. Bring confusion matrix
3. Add notes in streamlit App itself for demonstration
4. Create Readme

In [12]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.10.26.84
⠙your url is: https://twenty-islands-throw.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:24397 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙